In [16]:
!pip install requests tqdm xarray rioxarray netCDF4 shapely geopandas pandas matplotlib


In [17]:
import requests
import pandas as pd
from shapely.geometry import box
from datetime import datetime
from tqdm import tqdm
import os

In [28]:
# ---- User config ----
CLIENT_ID = "sh-051b99c9-b0d6-46ac-841c-6bb1f6bd6fed"
CLIENT_SECRET = "vvJZ5VC3fZ76wne9fLPnIRFs3RdEi2QF"

aoi = box(36.5, -1.5, 37.0, -1.0)   # Example: Nairobi area
start_date = "2021-01-01"
end_date = "2021-12-31"

# Create download folder
os.makedirs("s3_data", exist_ok=True)


In [29]:
import requests

def get_copernicus_token(client_id, client_secret):
    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials"
    }
    
    response = requests.post(url, data=data)
    response.raise_for_status()
    
    token = response.json().get("access_token")
    print("✅ Token acquired successfully!")
    return token

ACCESS_TOKEN = get_copernicus_token(CLIENT_ID, CLIENT_SECRET)


✅ Token acquired successfully!


In [34]:
import requests
import pandas as pd

def search_s3_slstr(aoi, start_date, end_date, platform="S3A"):
    """
    Search Sentinel-3 SLSTR Level-2 LST data in Copernicus Data Space.
    aoi: [lon_min, lat_min, lon_max, lat_max]
    """
    url = "https://catalogue.dataspace.copernicus.eu/resto/api/collections/SENTINEL-3/search.json"

    lon_min, lat_min, lon_max, lat_max = aoi

    # Build a valid WKT polygon (longitude/latitude order)
    wkt_geometry = (
        f"POLYGON(({lon_min} {lat_min}, {lon_min} {lat_max}, "
        f"{lon_max} {lat_max}, {lon_max} {lat_min}, {lon_min} {lat_min}))"
    )

    params = {
        "startDate": start_date,
        "completionDate": end_date,
        "platform": platform,
        "productType": "SL_2_LST___",
        "geometry": wkt_geometry,
        "maxRecords": 10,
    }

    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}",  # your OAuth token
        "User-Agent": "copernicus-api-client"
    }

    response = requests.get(url, params=params, headers=headers)
    print("🔗 Request URL:", response.url)

    if response.status_code == 401:
        raise Exception("❌ Unauthorized. Check your token or client credentials.")
    elif response.status_code == 403:
        raise Exception("🚫 Forbidden. Check your access rights to this dataset.")
    elif response.status_code == 400:
        raise Exception(f"⚠️ Bad request — check parameters or geometry format. ({response.text[:200]})")

    response.raise_for_status()

    features = response.json().get("features", [])
    if not features:
        print("⚠️ No products found for this AOI/date range.")
        return pd.DataFrame()

    return pd.DataFrame([
        {
            "id": f["id"],
            "title": f["properties"].get("title", ""),
            "startDate": f["properties"].get("startDate", ""),
            "platform": f["properties"].get("platform", ""),
            "productUrl": f["properties"]["services"]["download"]["url"]
        }
        for f in features
    ])


In [35]:
aoi = [36.5, -1.5, 37.0, -1.0]  # Kenya example
start_date = "2021-01-01"
end_date = "2021-12-31"

df_results = search_s3_slstr(aoi, start_date, end_date)
print("✅ Found products:", len(df_results))
display(df_results.head())


🔗 Request URL: https://catalogue.dataspace.copernicus.eu/resto/api/collections/SENTINEL-3/search.json?startDate=2021-01-01&completionDate=2021-12-31&platform=S3A&productType=SL_2_LST___&geometry=POLYGON%28%2836.5+-1.5%2C+36.5+-1.0%2C+37.0+-1.0%2C+37.0+-1.5%2C+36.5+-1.5%29%29&maxRecords=10
✅ Found products: 10


,id,title,startDate,platform,productUrl
0,1de17819-74ae-59b8-9e80-52c9c0b7ad25,S3A_SL_2_LST____20210301T191837_20210301T19203...,2021-03-01T19:18:37.222000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...
1,76f4d602-9a05-5ee2-beb4-35058aa38f13,S3A_SL_2_LST____20210528T193725_20210528T19392...,2021-05-28T19:37:25.295000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...
2,1469aef6-39a9-5518-8428-d3da816233ad,S3A_SL_2_LST____20210225T192222_20210225T19242...,2021-02-25T19:22:22.137000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...
3,afe34557-ecd1-5007-818f-71ae97cefb84,S3A_SL_2_LST____20210224T194833_20210224T19503...,2021-02-24T19:48:33.174000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...
4,912315c0-6e23-52c1-9f3e-6d33f34207fe,S3A_SL_2_LST____20210603T074424_20210603T07472...,2021-06-03T07:44:23.631000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...


In [ ]:
### Test1

In [42]:
# Authentication and setup
import requests

# --- Copernicus Data Space credentials ---
CLIENT_ID = "sh-051b99c9-b0d6-46ac-841c-6bb1f6bd6fed"
CLIENT_SECRET = "vvJZ5VC3fZ76wne9fLPnIRFs3RdEi2QF"

# --- Official token endpoint ---
TOKEN_URL = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

# Request access token
data = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials"
}

response = requests.post(TOKEN_URL, data=data)
response.raise_for_status()
ACCESS_TOKEN = response.json()["access_token"]

print("✅ Access token acquired successfully!")


✅ Access token acquired successfully!


In [43]:
import pandas as pd
from shapely.geometry import box

def search_s3_slstr(aoi_bounds, start_date, end_date, platform="S3A", token=None):
    """
    Searches the Copernicus Data Space catalogue for Sentinel-3 SLSTR products (LST).
    """
    url = "https://catalogue.dataspace.copernicus.eu/resto/api/collections/SENTINEL-3/search.json"
    
    geometry_wkt = f"POLYGON(({aoi_bounds[0]} {aoi_bounds[1]}, {aoi_bounds[0]} {aoi_bounds[3]}, {aoi_bounds[2]} {aoi_bounds[3]}, {aoi_bounds[2]} {aoi_bounds[1]}, {aoi_bounds[0]} {aoi_bounds[1]}))"

    params = {
        "startDate": start_date,
        "completionDate": end_date,
        "platform": platform,
        "productType": "SL_2_LST___",
        "geometry": geometry_wkt,
        "maxRecords": 10
    }

    headers = {"Authorization": f"Bearer {token}"} if token else {}

    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    features = response.json().get("features", [])

    if not features:
        print("⚠️ No products found for the given parameters.")
        return pd.DataFrame()

    df = pd.DataFrame([
        {
            "id": f["id"],
            "title": f["properties"]["title"],
            "startDate": f["properties"]["startDate"],
            "platform": f["properties"]["platform"],
            "productUrl": f["properties"]["services"]["download"]["url"]
        }
        for f in features
    ])

    return df


In [44]:
# Define an AOI (bounding box around Nairobi, Kenya)
aoi = [36.5, -1.5, 37.0, -1.0]  # lon_min, lat_min, lon_max, lat_max
start_date = "2021-01-01"
end_date = "2021-12-31"

df_results = search_s3_slstr(aoi, start_date, end_date, token=ACCESS_TOKEN)
print("✅ Found products:", len(df_results))
display(df_results.head())


✅ Found products: 10


,id,title,startDate,platform,productUrl
0,1de17819-74ae-59b8-9e80-52c9c0b7ad25,S3A_SL_2_LST____20210301T191837_20210301T19203...,2021-03-01T19:18:37.222000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...
1,1469aef6-39a9-5518-8428-d3da816233ad,S3A_SL_2_LST____20210225T192222_20210225T19242...,2021-02-25T19:22:22.137000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...
2,afe34557-ecd1-5007-818f-71ae97cefb84,S3A_SL_2_LST____20210224T194833_20210224T19503...,2021-02-24T19:48:33.174000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...
3,912315c0-6e23-52c1-9f3e-6d33f34207fe,S3A_SL_2_LST____20210603T074424_20210603T07472...,2021-06-03T07:44:23.631000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...
4,0eb76566-dd29-5321-8ca2-bf57fb652b26,S3A_SL_2_LST____20210220T071423_20210220T07172...,2021-02-20T07:14:23.459000Z,S3A,https://catalogue.dataspace.copernicus.eu/down...


In [46]:
import requests

CLIENT_ID = "sh-051b99c9-b0d6-46ac-841c-6bb1f6bd6fed"
CLIENT_SECRET = "vvJZ5VC3fZ76wne9fLPnIRFs3RdEi2QF"

TOKEN_URL = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

data = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials",
    "scope": "cdse-public"  # 👈 required for downloading Sentinel data
}

response = requests.post(TOKEN_URL, data=data)
response.raise_for_status()
ACCESS_TOKEN = response.json()["access_token"]

print("✅ Download-capable access token acquired!")


HTTPError: 400 Client Error: Bad Request for url: https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token